<a href="https://colab.research.google.com/github/jessicamarycooper/Backwards/blob/main/clustering_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from google.colab import drive
drive.mount('/content/Drive')
import torch
import os

!pip install update transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel, utils
import torch
import random
from matplotlib import pyplot as plt
%matplotlib inline
from IPython import display
import numpy as np
from tqdm import tqdm
from time import time
import json
utils.logging.set_verbosity_error()



Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

try:
    from transformers import GPT2Tokenizer, GPT2LMHeadModel, utils, AutoTokenizer, AutoModelForCausalLM
except:
    install('transformers')
    from transformers import GPT2Tokenizer, GPT2LMHeadModel, utils, AutoTokenizer, AutoModelForCausalLM

utils.logging.set_verbosity_error()

def load_all(model_name="gpt2", device='cpu'):
    cur_dir = os.listdir()
    
    if model_name + '_tokenizer' in str(cur_dir):
        print('Loading tokenizer...')
        tokenizer = torch.load(model_name + '_tokenizer')
    else:
        # Will have to change this line to support other models automatically
        print('Downloading tokenizer...')
        if 'gpt-j' in model_name:
            tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
        else:
            tokenizer = GPT2Tokenizer.from_pretrained(model_name, padding_side='left')
        torch.save(tokenizer, model_name + '_tokenizer')
    pad_token_id=tokenizer.eos_token_id


    if model_name + '_model' in str(cur_dir):
        print('Loading model...')
        model = torch.load(model_name + '_model').to(device)
    else:
        print('Downloading model...')

        if 'gpt-j' in model_name:
            model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B").to(device)
        else:
            model = GPT2LMHeadModel.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id).to(device)
        torch.save(model, model_name + '_model')
    model.eval()

    # 'word_embeddings' tensor gives emeddings for each token in the vocab for this model,
    # has shape (vocab_len, embedding_dimension) which in this case = (50257, 768)
    
    embeddings = model.transformer.wte.weight.to(device)
    if model_name + '_embeddings' not in str(cur_dir):
        torch.save(embeddings, model_name + '_embeddings')

    return model, embeddings, tokenizer

In [ ]:
model, embeddings, tokenizer = load_all("gpt2-xl", device=device)

vocab_len= embeddings.shape[0]
embedding_dim = embeddings.shape[-1] 

print(embeddings.shape)

# 'embeddings' tensor gives emeddings for each token in the vocab for this model

In [42]:
def normalise(x, min_max=[]):     
# normalises values of (array or tensor) x according to first (min) and second (max) values in list min_max. 
# This effectively defaults to [0,1] if the list doesn't contain exactly two elements. 
# The original code threw an error if min_max had length 1, so it's been changed slightly.

# First normalise x to [0,1]
    rnge = x.max() - x.min()
    if rnge > 0:
        x = (x - x.min())/rnge

# Now, if there's a min and max given in min_max list, multiply by difference and add minimum
    if len(min_max) > 1:
        rnge = min_max[1] - min_max[0]
        x = x * rnge + min_max[0]

    return x

def closest_tokens(emb, n=1):      
# This finds the n tokens in the vocabulary that are closest in the embedding space (in terms of Euclidean distance) to a given word embedding (‘emb’).
# Note that here 'emb' may or may not correspond to a token (i.e., it may or may not be a 'legal' embedding).
# Function returns a 4-tuple (list of the n tokens, list of their indices, list of their distances from emb, and list of their embedding vectors)
    torch.cuda.empty_cache()
    dists = torch.linalg.norm(embeddings - emb, dim=1)
    sorted_dists, ix = torch.sort(dists)	 
    # sorted_dists is a list of all embedding distances from 'emb', across entire vocab, sorted in increasing order, 
    # ix is a list of their corresponding 'vocab indices'
    tokens = [tokenizer.decode(i) for i in ix[:n]]
    # For each of the first n 'vocab indices' in ix, we decode it into the string version of the corresponding token. 
    # These strings then constitute the list 'tokens'.
    ixs = ix[:n]
    dists = sorted_dists[:n]
    embs = embeddings[ixs]  # Each of these n 'embeddings' is a tensor of shape (768,)
    return tokens, ixs, dists, embs  




In [ ]:
# Finding the closest token embeddings to the 'vocab centroid'

# For GPT-j we're going to remove all the rows from the 'embeddings' tensor which decode to something beginning "<|extratoken_"
# Get the token strings
token_strings = [tokenizer.decode([i]) for i in range(embeddings.shape[0])]

# Find the indices of the rows to keep
keep_indices = [i for i, token_str in enumerate(token_strings) if not token_str.startswith("<|extratoken_")]

# Keep only the rows with the desired token strings
filtered_embeddings = embeddings[keep_indices]


# calculate the mean vector of word_embeddings
vocab_centroid = filtered_embeddings.mean(dim=0)
 
# calculate the Euclidean distances from mean_vector to all the rows
distances = torch.norm(embeddings - vocab_centroid, dim=1)
 
# get the indices of the sorted distances
_, indices = torch.sort(distances)
 
# extract the word_embeddings with the smallest distances from the mean vector
closest_embeddings = tokenizer.decode(indices[:99]).split(' ')
 
for i in range(100):
   print("{:<7}{:35}{:<8}{:<8}{:<18}{:<12}".format('Token:', repr(tokenizer.decode(indices[i])), 'Index:', indices[i].item(), 'Distance:', distances[indices[i]].item()))


In [52]:
# k-means algorithm to find num_clusters clusters (clusters won't be of equal size), 
# also returns nearest_k tokens to each cluster centroid (unless the cluster size is less than nearest_k)

def kmeans(num_clusters, nearest_k=20):

    empty_cluster = True

    # euclidean distance threshold to break out of loop (may not be necessary as 0 seems to work fine)
    threshold = 0

    while empty_cluster == True:    # We're going to loop on random initialisation of centroids until all clusters are non-empty.
        # randomly generate num_clusters centroids, stack these into a tensor of shape (num_clusters, 768)
        # then normalise to vocab embedding span
        centroids = torch.rand(num_clusters, embedding_dim).to(device)
        centroids = normalise(centroids,[embeddings.min(dim=0)[0], embeddings.max(dim=0)[0]])

        distances = torch.cdist(embeddings, centroids, p=2)
        # This will be of shape (vocab_len, num_clusters), recording distances of all token embeddings from each of the centroids.
        closest_distance, closest_centroid = torch.min(distances, dim = -1)
        # These will be of shape (vocab_len,), recording the distance of each token embedding to nearest centroid, and the index of that centroid.

        clusters = []
        # We iterate over the centroids to build a list of clusters (tokens which share a common nearest centroid)
        for i in range(num_clusters):
            mask = closest_centroid == i  # This builds a Boolean mask over the complete set of tokens, True iff a token's nearest centroid is the ith.
            clusters.append(embeddings[mask]) 
            # embeddings[mask] is a subtensor of the shape (50257, 768) tensor involving only tokens whose nearest centroid is the ith.
            # This subtensor gets appended to the list 'clusters', which has num_clusters elements.

        cluster_sizes = [clusters[i].shape[0] for i in range(num_clusters)]  
        if 0 not in cluster_sizes:
            empty_cluster = False 

    # Now we have a set of num_clusters non-empty clusters, we begin iterating centroid positions:

    centroids_stable = False
    iterations = 0

    while centroids_stable == False:    # We're going to loop until the centroids stop moving.
        iterations += 1
        distances = torch.cdist(embeddings, centroids, p=2)
        # This will be of shape (vocab_len, num_clusters), recording distances of all token embeddings from each of the centroids.
        closest_distance, closest_centroid = torch.min(distances, dim = -1)
        # These will be of shape (vocab_len,), for each token embedding recording the distance to the nearest centroid, and the index of that centroid.

        clusters = []
        # We iterate over the centroids to build a list of clusters (tokens which share a common nearest centroid)
        for i in range(centroids.shape[0]):
            mask = closest_centroid == i  # This builds a Boolean mask over the complete set of tokens, True iff a token's nearest centroid is the ith.
            clusters.append(embeddings[mask]) 
            # embeddings[mask] is a subtensor of the (50257, 768) shape tensor involving only tokens nearest ith centroid.
            # This subtensor gets appended to the list 'clusters', which has num_clusters elements.

        new_centroids = []
        for i in range(num_clusters):
            new_centroids.append(clusters[i].mean(dim=0))
            # clusters[i].mean(dim=0) is the centroid of the set of vectors encoded in the tensor clusters[i]
            # These all get put in a list.

        new_centroids = torch.stack(new_centroids)  # stack the list into a tensor of shape (num_clusters, 768)

        # We now compute the euclidean distance between old and new centroids
        distance = torch.norm(new_centroids - centroids, dim=-1)

        if torch.max(distance) <= threshold:
            centroids_stable == True
            break
        centroids = new_centroids # if we're still outside the distance threshold, keep iterating
    
    for i in range(num_clusters):
        print('Cluster', i, 'contains', len(clusters[i]), 'embeddings.\n')
        #Now we're going to find the nearest k tokens in the cluster (give them + indices) and list them all with distances
        #If the cluster size k < 20, we'll just use the nearest k tokens

        k = clusters[i].shape[0]
        if k < nearest_k:
            nearest_k = k
        print('The nearest', nearest_k, 'tokens in cluster', i, 'to the cluster centroid:')
        cluster = clusters[i]
        centroid = centroids[i].unsqueeze(0) # adding a dimension
        cluster_distances = torch.norm(cluster - centroid, dim=1)
        top_k, top_k_indices = torch.topk(cluster_distances, k, largest=False, sorted=True)
        for j in range(nearest_k):
            embedding = clusters[i][top_k_indices[j].item()]
            distances = torch.norm(embeddings - embedding, dim=1)
            index = torch.argmin(distances)
            print("{:<7}{:35}{:<8}{:<8}{:<11}{:<15}".format('     Token:', repr(tokenizer.decode(index)), 'Index:', index.item(), 'Distance:', top_k[j].item()))
        if i < num_clusters - 1:
            print('')

        #Then we're going to find the nearest k tokens in the entire vocab (give them + indices)  and list them all with distances
        print('The nearest', nearest_k, 'tokens in the entire vocabulary to the cluster centroid:')
        centroid = centroids[i].unsqueeze(0) # adding a dimension
        vocab_distances = torch.norm(embeddings - centroid, dim=1)
        top_k, top_k_indices = torch.topk(vocab_distances, k, largest=False, sorted=True)
        for j in range(nearest_k):
            embedding = embeddings[top_k_indices[j].item()]
            distances = torch.norm(embeddings - embedding, dim=1)
            index = torch.argmin(distances)
            print("{:<7}{:35}{:<8}{:<8}{:<11}{:<15}".format('     Token:', repr(tokenizer.decode(index)), 'Index:', index.item(), 'Distance:', top_k[j].item()))
        if i < num_clusters - 1:
            print('\n')

    return centroids

In [ ]:
kmeans(24, 33)

In [ ]:
# After getting bogged down in trying to reinvent the wheel, keeping track of recursively reshuffled indices, 
# I asked ChatGPT3 and it wrote this (only the kmeans = KMeans... line had to be adjusted due to CPU/GPU issues).
# It's using Scikit-learn or sklearn, which is a popular python package for this kind of thing.

# Unless we can get this to run on some faster hardware, this is futile. It will take all day to find these centroids.
# ChatGPT suggested using cuml or rapidsai, but pip installation failed for both of these.


def kkmeans(k):

    from sklearn.cluster import KMeans

    # k = number of clusters

    # Apply k-means clustering
    print('Applying k-means clustering...')
    kmeans = KMeans(n_clusters=k, random_state=0).fit(embeddings.detach().cpu().numpy())
    
    # Get the cluster assignments for each point
    cluster_assignments = kmeans.labels_
    print('Getting the cluster assignments for each point...')

    # Count the number of points in each cluster
    cluster_sizes = np.bincount(cluster_assignments)
    print('Counting the number of points in each cluster...')

    # Initialize an array to store the new cluster assignments
    new_assignments = np.zeros(cluster_assignments.shape)
    print('Initializing an array to store the new cluster assignments...')

    # Target cluster size
    target_size = len(embeddings) // k
    print('Target cluster size =', target_size)

    # Initialize a list to store the indices of the points in each cluster
    clusters = [[] for _ in range(k)]
    for i, c in enumerate(cluster_assignments):
        clusters[c].append(i)

    # Reassign points to achieve equal cluster sizes
    for c in range(k):
        while len(clusters[c]) > target_size:
            # Find the point in the current cluster that is closest to another cluster
            min_distance = float('inf')
            min_point = None
            min_cluster = None
            for i in clusters[c]:
                print('Working on cluster ', c, ' size =', len(clusters[c]), '; token embedding: ', i)
                point = embeddings[i]
                for j in range(k):
                    if j == c:
                        continue
                    centroid = kmeans.cluster_centers_[j]
                    distance = torch.norm(point.to(device) - torch.from_numpy(centroid).to(device))
                    if distance < min_distance:
                        min_distance = distance
                        min_point = i
                        min_cluster = j
            # Move the point to the closest cluster
            clusters[c].remove(min_point)
            clusters[min_cluster].append(min_point)
            new_assignments[min_point] = min_cluster

    # Get the new cluster assignments
    kmeans.labels_ = new_assignments

    # Get the centroids of each cluster
    centroids = kmeans.cluster_centers_
    centroids = torch.from_numpy(centroids)

    # return the centroids